缺失值
各特徵是否有問題


In [1]:
# import numpy as np
# import pandas as pd
# import os
# import copy
# import matplotlib.pyplot as plt
# %matplotlib inline
# from tqdm import tqdm_notebook
# from sklearn.preprocessing import StandardScaler
# from sklearn.svm import NuSVR, SVR
# from sklearn.metrics import mean_absolute_error
# pd.options.display.precision = 15
# from collections import defaultdict
# import lightgbm as lgb
# import xgboost as xgb
# import catboost as cat
# # import timeb
# from collections import Counter
# import datetime
# from catboost import CatBoostRegressor
# from sklearn.preprocessing import LabelEncoder
# from sklearn.model_selection import StratifiedKFold, KFold, RepeatedKFold, GroupKFold, GridSearchCV, train_test_split, TimeSeriesSplit, RepeatedStratifiedKFold
# from sklearn import metrics
# from sklearn.metrics import classification_report, confusion_matrix
# from sklearn import linear_model
# import gc
# import seaborn as sns
# import warnings
# # warnings.filterwarnings("ignore")
# from bayes_opt import BayesianOptimization
# import eli5
# import shap
# from IPython.display import HTML
# import json
# import altair as alt
# from category_encoders.ordinal import OrdinalEncoder
# import networkx as nx
# import matplotlib.pyplot as plt
# %matplotlib inline
# from typing import List

# import os
# import time
# import datetime
# import json
# import gc
# from numba import jit

# import numpy as np
# import pandas as pd

# import matplotlib.pyplot as plt
# import seaborn as sns
# from tqdm import tqdm_notebook

# import lightgbm as lgb
# import xgboost as xgb
# from catboost import CatBoostRegressor, CatBoostClassifier
# from sklearn import metrics
# from typing import Any
# from itertools import product
# pd.set_option('max_rows', 500)
# import re
# from tqdm import tqdm
# from joblib import Parallel, delayed

In [2]:
import numpy as np
import pandas as pd
from IPython.display import display


from utils import kaggle_util

In [3]:
# https://www.kaggle.com/robikscube/2019-data-science-bowl-an-introduction

import seaborn as sns
sns.set_style("whitegrid")
my_pal = sns.color_palette(n_colors=10)

## https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.get_option.html#pandas.get_option
pd.set_option('display.max_columns',100)
pd.set_option('display.max_rows',100)

pd.set_option('display.max_rows',10000)

In [4]:
## 刪掉時間太少的gs

def del_gs_by_time(train, del_time_th=10):
    """
    del_time_th: delete game_session less than 10s
    """
    print('\nDeleteing gs by time ... ...')
    ## 看兩次game_session時間差異
    iid_gp = train.groupby(['installation_id'])
    del_gs = []
    for iid, gp in iid_gp:
        x = gp.groupby(['game_session'],sort=False)['timestamp','type'].first()
        x2 = x.timestamp
        x2 = (x2 - x2.shift(1)).astype('timedelta64[s]')
        del_gs += list(x.index.values[(x2<del_time_th) & (x['type']!='Assessment')])

    print(len(del_gs))

    ## 直接看game_time
    train2 = train[(train['type']=='Activity') | (train['type']=='Game')]
    x = train2.groupby(['game_session'])['game_time'].last()
    del_gs2 = x[x<del_time_th*1000].index.values
    print(len(del_gs2))

    del train2
    del_gs = set(del_gs)
    del_gs.update(set(del_gs2))

    print(len(del_gs))
    del_id = train.game_session.isin(del_gs)
    train2 = train.loc[~del_id]
    return train2

In [5]:
# read data
train, test, train_labels, specs, sample_submission = kaggle_util.read_data()
train.timestamp = pd.to_datetime(train.timestamp)
test.timestamp = pd.to_datetime(test.timestamp)
train1 = del_gs_by_time(train, del_time_th=10)
test1 = del_gs_by_time(test, del_time_th=10)
print(train.shape)
print(train1.shape)
print(test.shape)
print(test1.shape)

train2, test2, train_labels2 = train1.copy(), test1.copy(), train_labels.copy()

# get usefull dict with maping encode, we implement "label encoding" on category features
train2, test2, train_labels2, win_code, list_of_user_activities, list_of_event_code,\
activities_labels, assess_titles, list_of_event_id, all_title_event_code, activities_map = kaggle_util.encode_title(train2, test2, train_labels2)

# tranform function to get the train and test set
reduce_train, reduce_psuedo_train, reduce_test, categoricals = kaggle_util.get_train_and_test(train2, test2, win_code, list_of_user_activities, list_of_event_code,\
activities_labels, assess_titles, list_of_event_id, all_title_event_code, activities_map, create_psuedo_assessment=False)


Reading train.csv file....
Training.csv file have 11341042 rows and 11 columns
Reading test.csv file....
Test.csv file have 1156414 rows and 11 columns
Reading train_labels.csv file....
Train_labels.csv file have 17690 rows and 7 columns
Reading specs.csv file....
Specs.csv file have 386 rows and 3 columns
Reading sample_submission.csv file....
Sample_submission.csv file have 1000 rows and 2 columns

Deleteing gs by time ... ...
2107
7423
9451

Deleteing gs by time ... ...
171
716
881
(11341042, 11)
(11305461, 11)
(1156414, 11)
(1153386, 11)



add_time_diff_hour ...
add_time_diff_hour ...
add_session_cnt ... ...
add_session_cnt ... ...


In [6]:
# reduce_train.to_csv('../data/preprocess/'+'train895.csv', index=False)
# reduce_test.to_csv('../data/preprocess/'+'test895.csv', index=False)
reduce_train.to_csv('../data/preprocess/'+'train_0106.csv', index=False)
reduce_psuedo_train.to_csv('../data/preprocess/'+'psuedo_train_0106.csv', index=False)
reduce_test.to_csv('../data/preprocess/'+'test_0106.csv', index=False)


In [ ]:
reduce_train.session_cnt_30min.value_counts()

In [ ]:
## 檢查有沒有缺失值
reduce_train.columns[(reduce_train.isnull().sum()!=0)]

In [ ]:
## 以使用者觀察
iid_list = train.installation_id.unique()
train[train['installation_id']==iid_list[10]].groupby(['game_session'])['title','type','timestamp'].first().sort_values('timestamp')

In [ ]:
## 找重度使用者
iid_gs_cnt = train.groupby(['installation_id'])['game_session'].nunique()
tmp = iid_gs_cnt[iid_gs_cnt>300]
print(tmp)

##計算重度使用者比例
print(train.game_session.nunique())
print(np.sum(tmp))

In [ ]:
train.game_session.iloc[0]+'a'

In [ ]:
## 觀察重度使用者
x = train[train['installation_id']=='08987c08'].groupby(['game_session'])['title','type','timestamp'].first().sort_values('timestamp')
x

In [ ]:
x = train[train['installation_id']=='063e3e02'].groupby(['game_session'])['title','type','timestamp'].first().sort_values('timestamp')
display(x)
    

In [ ]:
cnt=0
for id, gp in reduce_train.groupby(['installation_id']):
    cnt+=1
    if (cnt)>500:
        break
    print(id)
#     display(gp.iloc[:,-5:])
    gp['title'] = gp['title'].map(activities_labels)
    display((gp.loc[:,['title','accuracy_group']]))


In [ ]:
## 檢查多人使用的情形
## 查看多人使用的測資準確率多少?

def describe_game_session(x,th=200):
    total_gs = x.game_session.nunique()
    id_gs = x.groupby(['installation_id'])['game_session'].nunique()
    id_gs = id_gs[id_gs>th]
    print(total_gs)
    print(np.sum(id_gs))
    print(id_gs)
    
describe_game_session(train,200)
describe_game_session(test,200)

In [ ]:
## 依據type觀察event
display(train[train['type']=='Clip'].iloc[:500])
display(train[train['type']=='Activity'].iloc[:500])
display(train[train['type']=='Game'].iloc[:500])
display(train[train['type']=='Assessment'].iloc[:500])


In [ ]:
## 觀察有哪些category資料在train上出現 test沒有(就是指太少出現)，應改掉
## event id

print(train.event_id.nunique())
print(test.event_id.nunique())

print(train.event_code.nunique())
print(test.event_code.nunique())

x = test.groupby(['event_id'])['game_session'].count()
x = test.groupby(['event_code'])['game_session'].count()



## Garbage

In [ ]:
## 檢查那些沒有label的使用者，有沒有足夠的使用紀錄
iid_list = train.installation_id.unique()
iid_list2 = train_labels.installation_id.unique()
cnt=0
for iid in iid_list:
    if iid not in iid_list2:
        cnt+=1
        print(train[train['installation_id']==iid].game_session.nunique())
        if cnt>10:
            break


In [ ]:
## 依使用者觀察，以game_session為單位

# id_all = train.groupby(['installation_id'])
# cnt=0
# for id1, x in id_all:
#     cnt+=1
#     if cnt>10:
#         break
#     print(id1)
#     x2 = x.groupby(['game_session']).first()
#     print(x2.loc[:,['title','type','world','timestamp']])

# # reduce_train.loc[:,['installation_id','timestamp']]

In [ ]:
a={1:0,2:2}
np.sum(a.values())
sum(a.values())

In [ ]:
a={1:0,2:2}
features = dict()
features[1] = a[1]
features
a[1]=4
features


In [ ]:
list(filter(lambda x: x > 0, a.values()))
np.sum([])

In [ ]:
pd.Series(assess_to_title).to_csv('../data/preprocess/assess_to_title.csv')

In [ ]:
x = pd.read_csv('../data/raw_data/media_sequence.csv')
assess_to_title = dict()
tmp = []
for i in range(1,x.shape[0]):
    tmp.append(x.loc[i,'title'])
    if x.loc[i,'type'] == 'Assessment':
        assess_to_title['{}'.format(x.loc[i,'title'])]=tmp
        tmp = []
assess_to_title